<a href="https://colab.research.google.com/github/hyojunyee/kita_2404/blob/main/m5_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/ML_pytorch_%EC%8B%A4%EC%8A%B5_0829.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q. mnist 데이터 셋에 대해서 Pytorch를 적용하여 모델 구성 변경, 조기 학습 중단을 수행하고 Best Model을 저장한 후 다시 불러와서 테스트 데이터로 평가한 결과를 출력하세요

In [1]:
# torch.max(outputs, 1)
import torch

# 예시 출력 텐서
outputs = torch.tensor([[0.1, 0.3, 0.6], [0.2, 0.7, 0.1]])

# 가장 큰 값을 가지는 클래스 인덱스를 추출
_, predicted = torch.max(outputs, 1) # 각 행에 대해 최대값

print(predicted)  # tensor([2, 1])

tensor([2, 1])


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

# MNIST 데이터셋을 위한 전처리 과정 정의
transform = transforms.Compose([
    transforms.ToTensor(),                  # 이미지를 PyTorch 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))    # 이미지를 평균 0.5, 표준편차 0.5로 정규화하여 [-1, 1] 범위로 조정
])

# MNIST 데이터셋 로드
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)       # test dataset이므로 train=False

# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size = int(0.8 * len(train_dataset))      # 훈련 세트 크기를 전체의 80%로 설정
val_size = len(train_dataset) - train_size      # 검증 세트 크기 계산
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# 데이터 로더 생성
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 아키텍처 정의
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)    # 컨볼루션 레이어 정의(입력 채널 1, 출력 채널 20, 커널 크기 5)
        self.pool = nn.MaxPool2d(2, 2)      # 멕스풀링 레이어 정의 (2x2 풀링)
        self.flatten = nn.Flatten()         # 텐서 평탄화
        self.fc1 = nn.Linear(2880, 50)      # 완전 연결 레이어 (입력크기 2880, 출력크기 50)
        self.fc2 = nn.Linear(50, 10)        # 출력 레이어 (클래스 수 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))    # ReLU 활성화 함수 적용 후 맥스풀링
        x = self.flatten(x)                     # 평탄화
        x = F.relu(self.fc1(x))                 # ReLU 활성화 함수 적용
        x = self.fc2(x)                         # 최종 출력 레이어
        return x

model = MyModel()       # 모델 인스턴스 생성

# 손실 함수 및 최적화 알고리즘 지정
criterion = nn.CrossEntropyLoss()       # 크로스 엔트로피 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)    # SGD 최적화 알고리즘

# 모델 훈련
best_val_loss = float('inf')    # 검증 손실을 추적하기 위한 변수 초기화
patience, trials = 5, 0         # 조기 학습 중단을 위한 변수 초기화
num_epochs = 20                 # 에폭 수 설정
for epoch in range(num_epochs):
    model.train()               # 모델을 훈련 모드로 설정
    running_loss = 0.0          # 손실을 추적하기 위한 변수 초기화
    for inupts, labes in train_loader:
        optimizer.zero_grad()               # 그래디언트 초기화
        outputs = model(inupts)             # 모델에 입력을 전달하여 출력 계산(모델을 통한 순전파)
        loss = criterion(outputs, labes)    # 손실 계산
        loss.backward()                     # 역전파 수행하여 그래디언트 계산
        optimizer.step()                    # 가중치(파라미터) 업데이트
        running_loss += loss.item()         # 손실 누적 계산

    # 검증 단계
    val_loss = 0.0
    model.eval()                # 모델을 평가 모드로 설정
    with torch.no_grad():       # 그래디언트 계산 비활성화
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(val_loader)     # 평균 검증 손실 계산
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss}")

    # 검증 손실이 개선되었는지 확인하고 모델 저장
    if val_loss < best_val_loss:
        print(f"Validation Loss Decreased {best_val_loss:.6f} ---> {val_loss:.6f}. \t Saving The Model...")
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_model.pth')    # 모델 저장
    else:
        trials += 1
        if trials >= patience:
            print("Early stopping triggered!")
            break
# 최고의 모델을 불러와서 평가
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# 테스트 데이터로 모델 평가
correct = 0
total = 0
with torch.no_grad():       # 그래디언트 계산 비활성화
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)   # torch.max(input, dim), 'dim'은 차원을 의미, 가장 높은 값을 갖는 클래스를 예측하겠다는 의미
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 정확도 계산 및 출력
accuracy = 100 * correct / total
print(f"Accuracy of the model on the 10000 test images: {accuracy}%")

Epoch 1/20, Training Loss: 0.6940400156080723, Validation Loss: 0.29293933439444986
Validation Loss Decreased inf ---> 0.292939. 	 Saving The Model...
Epoch 2/20, Training Loss: 0.2457382708688577, Validation Loss: 0.21058484733580274
Validation Loss Decreased 0.292939 ---> 0.210585. 	 Saving The Model...
Epoch 3/20, Training Loss: 0.1754001368979613, Validation Loss: 0.16291851566192952
Validation Loss Decreased 0.210585 ---> 0.162919. 	 Saving The Model...
Epoch 4/20, Training Loss: 0.1360593303044637, Validation Loss: 0.1295022857276366
Validation Loss Decreased 0.162919 ---> 0.129502. 	 Saving The Model...
Epoch 5/20, Training Loss: 0.11120735496779283, Validation Loss: 0.10794336425061239
Validation Loss Decreased 0.129502 ---> 0.107943. 	 Saving The Model...
Epoch 6/20, Training Loss: 0.09537954062968493, Validation Loss: 0.1009589708013896
Validation Loss Decreased 0.107943 ---> 0.100959. 	 Saving The Model...
Epoch 7/20, Training Loss: 0.08321228069315353, Validation Loss: 0.08

<ipython-input-4-2259c93ecb62>:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Accuracy of the model on the 10000 test images: 98.37%


- torch: 이것은 메인 PyTorch 라이브러리입니다. 여기에는 GPU를 통한 가속 텐서 계산 지원, 신경망 훈련을 용이하게 하는 자동 차별화, 모델 구축 및 훈련을 위한 다양한 유틸리티가 포함됩니다.
- torch.nn: 레이어, 활성화 함수, 손실 함수와 같은 신경망의 구성 요소를 제공하는 PyTorch의 하위 모듈입니다. 신경망의 아키텍처를 정의하는 데 필수적입니다.
- torch.nn.function: 이 모듈에는 torch.nn 레이어에서 사용되는 기능이 포함되어 있습니다. 입력 데이터 및 가중치에 이러한 함수를 직접 사용할 수 있으므로 일부 작업에 더 많은 유연성을 제공합니다. 여기에는 활성화, 손실 계산 및 상태(즉, 가중치)를 유지하지 않는 다양한 기타 작업을 위한 함수가 포함됩니다.
- torch.optim: 이 하위 모듈은 SGD(Stochastic Gradient Descent), Adam 등과 같은 신경망 훈련을 위한 최적화 알고리즘을 제공합니다. 이러한 최적화 프로그램은 계산된 기울기를 기반으로 네트워크의 가중치를 업데이트하는 데 사용됩니다.
- torchvision: 이미지 데이터 작업을 위한 유틸리티를 제공하는 PyTorch 프로젝트의 패키지입니다. 여기에는 사전 정의된 데이터세트(예: MNIST, CIFAR10, FashionMNIST), 모델 아키텍처(예: ResNet, AlexNet) 및 전처리를 위한 일반적인 이미지 변환이 포함됩니다.
- torchvision.transforms: 일반적인 이미지 변환을 제공하는 torchvision 내의 모듈입니다. 이는 이미지를 신경망에 공급하기 전에 데이터 증대 및 이미지 전처리에 사용될 수 있습니다. 예로는 크기 조정, 정규화, 텐서로 변환 등이 있습니다.
- SubsetRandomSampler: 교체 없이 데이터세트에서 요소를 무작위로 샘플링하는 데 사용되는 도구입니다. 데이터 세트를 훈련 및 검증/테스트 세트로 분할하거나 모델 훈련을 위해 사용자 정의 데이터 샘플링 전략을 구현하려는 경우에 특히 유용

Q. PyTorch를 사용하여 FashionMNIST 데이터세트에 대한 분류 모델링 및 평가를 다음과 같은 단계로 수행하세요.
- 1단계: 신경망 모델 정의 - Conv layer 추가
- 2단계: FashionMNIST 데이터셋 로드
- 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
- 4단계: 조기 종료를 포함한 모델 학습 및 best model 저장
- 5단계: best model을 로드하고 테스트 데이터셋으로 평가

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

# 1. 신경망 모델 정의 - Conv layer 추가
# CNN 모델 정의
class FashionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   # 1채널 입력, 32채널 출력, 3x3 커널
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 32채널 입력, 64채널 출력, 3x3 커널
        self.pool = nn.MaxPool2d(2, 2)                 # 2x2 맥스풀링
        self.fc1 = nn.Linear(64 * 7 * 7, 128)          # 64채널, 7x7 크기 입력, 128 출력
        self.fc2 = nn.Linear(128, 10)                  # 128 입력, 10 클래스 출력

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 첫 번째 Conv 레이어와 맥스풀링
        x = self.pool(F.relu(self.conv2(x)))  # 두 번째 Conv 레이어와 맥스풀링
        x = x.view(-1, 64 * 7 * 7)            # 평탄화
        x = F.relu(self.fc1(x))               # 첫 번째 완전 연결 레이어
        x = self.fc2(x)                       # 출력 레이어
        return x


# 2. FashionMNIST 데이터셋 로드
# 데이터 전처리 정의
transform = transforms.Compose([
    transforms.ToTensor(),                  # 이미지를 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))    # 이미지를 [-1, 1] 범위로 정규화
])

# 데이터셋 로드
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# 데이터 로더 생성
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 3. 네트워크, 손실 함수, 최적화 알고리즘 초기화
model = FashionCNN()                    # 모델 인스턴스 생성
criterion = nn.CrossEntropyLoss()       # 손실 함수 정의
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # 최적화 알고리즘 정의


# 4. 조기 종료를 포함한 모델 학습 및 best model 저장
best_val_loss = float('inf')    # 최적 검증 손실 초기화
patience, trials = 5, 0         # 조기 종료 변수 초기화
num_epochs = 20                 # 에폭 수 설정

for epoch in range(num_epochs):
    model.train()               # 모델을 훈련 모드로 설정
    running_loss = 0.0          # 훈련 손실 추적 변수 초기화
    for inputs, labels in train_loader:
        optimizer.zero_grad()               # 그래디언트 초기화
        outputs = model(inputs)            # 모델에 입력 전달
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()                     # 역전파
        optimizer.step()                    # 가중치 업데이트
        running_loss += loss.item()         # 손실 누적

    # 검증 단계
    val_loss = 0.0
    model.eval()                # 모델을 평가 모드로 설정
    with torch.no_grad():       # 그래디언트 계산 비활성화
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(val_loader)     # 평균 검증 손실 계산
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss}")

    # 검증 손실이 개선되었는지 확인하고 모델 저장
    if val_loss < best_val_loss:
        print(f"Validation Loss Decreased {best_val_loss:.6f} ---> {val_loss:.6f}. \t Saving The Model...")
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_model_fashion.pth')    # 모델 저장
    else:
        trials += 1
        if trials >= patience:
            print("Early stopping triggered!")
            break


# 5. best model을 로드하고 테스트 데이터셋으로 평가
# 최고의 모델을 불러와서 평가
model.load_state_dict(torch.load('best_model_fashion.pth'))
model.eval()

# 테스트 데이터로 모델 평가
correct = 0
total = 0
with torch.no_grad():       # 그래디언트 계산 비활성화
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)   # 예측값 계산
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the model on the 10000 test images: {100 * correct / total}%")

100%|██████████| 26421880/26421880 [00:06<00:00, 3811788.91it/s] 


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 308901.07it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5574100.22it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 12466672.63it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch 1/20, Training Loss: 1.047073925336202, Validation Loss: 0.5704078032298291
Validation Loss Decreased inf ---> 0.570408. 	 Saving The Model...
Epoch 2/20, Training Loss: 0.5362727802991867, Validation Loss: 0.4737735224214006
Validation Loss Decreased 0.570408 ---> 0.473774. 	 Saving The Model...
Epoch 3/20, Training Loss: 0.46314132169882455, Validation Loss: 0.4598541316041287
Validation Loss Decreased 0.473774 ---> 0.459854. 	 Saving The Model...
Epoch 4/20, Training Loss: 0.4201144840121269, Validation Loss: 0.40168853429086665
Validation Loss Decreased 0.459854 ---> 0.401689. 	 Saving The Model...
Epoch 5/20, Training Loss: 0.39400856286287306, Validation Loss: 0.37509245266939734
Validation Loss Decreased 0.401689 ---> 0.375092. 	 Saving The Model...
Epoch 6/20, Training Loss: 0.3745802163084348, Validation Loss: 0.3770736472562273
Epoch 7/20, Training Loss: 0.35913575494289396, Validat

<ipython-input-7-8f67a10448f9>:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_fashion.pth'))


Accuracy of the model on the 10000 test images: 89.49%


강사님ver.

In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

# 1단계: 신경망 모델 정의
# Net 클래스는 nn.Module을 상속받아 만들어진 사용자 정의 신경망 모델로, FashionMNIST 데이터셋의 이미지 분류를 위해 설계
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 첫 번째 합성곱 레이어: 입력 채널 1개, 출력 채널 6개, 커널 크기 5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 두 번째 합성곱 레이어: 입력 채널 6개, 출력 채널 16개, 커널 크기 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        # 전결합 레이어
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 16개의 채널과 4x4 이미지 크기
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 첫 번째 합성곱, ReLU 활성화, 맥스 풀링
        x = self.pool(F.relu(self.conv1(x)))
        # 두 번째 합성곱, ReLU 활성화, 맥스 풀링
        x = self.pool(F.relu(self.conv2(x)))
        # 입력 데이터를 1차원으로 펼침
        x = self.flatten(x)
        # x = x.view(-1, 16 * 4 * 4)
        # 전결합 레이어와 ReLU 활성화 함수 적용
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 마지막 레이어 출력
        x = self.fc3(x)
        return x

# 2단계: FashionMNIST 데이터셋 로드
# torchvision 라이브러리를 사용하여 FashionMNIST 데이터셋을 다운로드하고, 데이터를 전처리하기 위한 변환(transform)을 설정하는 과정

# transforms.Compose는 여러 전처리 단계를 하나로 묶어주는 역할
# transforms.ToTensor(): 이미지를 PyTorch 텐서로 변환. 이미지의 픽셀 값 범위가 0에서 255 사이의 정수에서 0.0에서 1.0 사이의 부동소수점으로 변경
# 모든 채널의 평균을 0.5로, 표준편차를 0.5로 설정합니다. 이는 데이터의 범위를 대략적으로 -1.0에서 1.0 사이로 조정
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# FashionMNIST 데이터셋을 다운로드하고, 지정된 변환을 적용하여 데이터를 준비하는 함수
train_val_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 및 검증 분할을 위한 데이터 인덱스 생성
dataset_size = len(train_val_dataset) # 훈련 및 검증 데이터셋의 전체 크기
indices = list(range(dataset_size)) # 데이터셋 내의 모든 샘플에 대한 인덱스를 포함
validation_split = 0.1 # 검증 세트로 사용될 데이터의 비율
split = int(np.floor(validation_split * dataset_size)) # 검증 세트의 크기를 계산
np.random.shuffle(indices) # 훈련 및 검증 세트가 데이터셋의 특정 부분에 치우치지 않도록 하기 위함
train_indices, val_indices = indices[split:], indices[:split] # 섞인 인덱스를 사용하여 훈련 세트와 검증 세트의 인덱스를 분할

# PT 데이터 샘플러 및 로더 생성
# 훈련 세트와 검증 세트에 대한 데이터 로더를 설정하고, 테스트 세트에 대한 데이터 로더를 별도로 설정하는 과정입니다.
# 이러한 데이터 로더들은 모델 학습, 검증, 테스트 과정에서 배치 단위로 데이터를 로드하는 데 사용
train_sampler = SubsetRandomSampler(train_indices) # train_indices에 해당하는 훈련 데이터의 인덱스를 무작위로 샘플링하는 샘플러를 생성
val_sampler = SubsetRandomSampler(val_indices) # val_indices에 해당하는 검증 데이터의 인덱스로부터 데이터를 무작위로 샘플링하는 샘플러를 생성
# 배치 크기 4로 로드하는 훈련 데이터 로더를 생성(# 데이터의 무작위 샘플링을 수행)
trainloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=train_sampler)
valloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=val_sampler)
# shuffle=False 인자를 통해 셔플링 없이 순서대로 데이터를 로드. 테스트 과정에서는 데이터의 순서가 결과에 영향을 미치지 않으므로 셔플링을 수행하지 않습니다.
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

# 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
net = Net() #  클래스의 인스턴스를 생성하여 net 변수에 할당
criterion = nn.CrossEntropyLoss() # 멀티클래스 분류 문제에서 널리 사용되는 손실 함수로, 모델의 예측값과 실제 타겟값 사이의 차이를 측정
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 모멘텀은 최적화 과정에서 이전 그래디언트의 방향을 고려, 파라미터 업데이트 시 관성을 부여

# 4단계: 조기 종료를 포함한 모델 학습
patience = 5 # 검증 손실이 개선되지 않을 때, 훈련을 계속 진행하기 전에 기다릴 에폭 수를 의미
patience_counter = 0
best_val_loss = np.Inf # 최고의 검증 손실 값을 무한대로 초기화. 훈련 과정에서 검증 손실이 이전에 기록된 최소 손실보다 낮아지면 업데이트되는 값

# 모델을 에폭 단위로 반복 훈련시키면서, 각 배치의 손실을 계산하고 모델 파라미터를 업데이트하는 기본적인 훈련 과정을 구현
for epoch in range(20):  # 데이터셋을 여러 번 반복
    net.train()  # 모델을 학습 모드로 설정. 이는 모델 내의 특정 레이어(예: 드롭아웃, 배치 정규화 등)가 훈련 시와 평가 시 다르게 동작해야 할 때 필요
    running_loss = 0.0 # 현재 에폭의 총 손실을 계산하기 위해 실행 손실을 0으로 초기화
    for i, data in enumerate(trainloader, 0): # '0'은 열거의 시작 인덱스를 지정
        inputs, labels = data
        optimizer.zero_grad() # 최적화를 수행하기 전에 모델의 그래디언트를 0으로 초기화
        outputs = net(inputs) # 현재 배치의 입력 데이터를 모델에 전달하여 예측값을 계산
        loss = criterion(outputs, labels) # 모델의 예측값과 실제 레이블 간의 손실을 계산
        loss.backward() # 손실 함수의 그래디언트를 역전파합니다. 이 과정에서 모델 파라미터에 대한 손실의 미분값이 계산
        optimizer.step() # 계산된 그래디언트를 사용하여 모델의 파라미터를 업데이트
        running_loss += loss.item() # 현재 배치의 손실을 실행 손실에 누적. 이를 통해 전체 에폭의 평균 손실을 계산

    # 검증 단계
    net.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0 # 검증 손실을 계산하기 위한 변수를 0으로 초기화
    with torch.no_grad(): # 이 블록 내에서는 그래디언트 계산을 비활성화. 평가 모드에서는 모델을 업데이트하지 않으므로, 그래디언트를 계산할 필요가 없습니다
        for inputs, labels in valloader: # 검증 데이터 로더(valloader)에서 배치 단위로 데이터를 로드하여 반복
            outputs = net(inputs)
            loss = criterion(outputs, labels) # criterion은 손실 함수로, 모델의 성능을 측정하는 기준
            val_loss += loss.item() # 누적된 검증 손실을 검증 데이터 배치의 총 수로 나누어 평균 검증 손실을 계산
    running_loss /= len(trainloader)
    val_loss /= len(valloader)
    print(f'에폭 {epoch + 1}, 훈련 손실: {running_loss}, 검증 손실: {val_loss}')

    # 조기 종료 체크
    if val_loss < best_val_loss: # 현재 에폭에서 계산된 검증 손실(val_loss)이 이전에 기록된 최소 검증 손실(best_val_loss)보다 낮은지 확인
        print(f'검증 손실이 감소하였습니다. ({best_val_loss:.6f} 에서 {val_loss:.6f}로). 모델 저장 중...')
        torch.save(net.state_dict(), '/content/drive/MyDrive/kdt_240424/m6_딥러닝/dataset/model/best_model.pth')
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('조기 종료 발동.')
            break

# 5단계: 최고의 모델을 로드하고 테스트 데이터셋으로 평가
# torch.load 함수는 지정된 경로에서 모델 파라미터를 불러오며, load_state_dict 메서드를 사용하여 이 파라미터를 현재 net 모델에 로드
net.load_state_dict(torch.load('/content/drive/MyDrive/kdt_240424/m6_딥러닝/dataset/model/best_model.pth'))
correct = 0 # 정확히 분류된 샘플의 수를 세기 위한 변수
total = 0 # 테스트셋의 전체 샘플 수를 세기 위한 변수
with torch.no_grad():
    for data in testloader: # 테스트 데이터셋을 배치 단위로 순회
        images, labels = data
        outputs = net(images) # 현재 배치의 이미지를 모델에 전달하여 예측값을 계산
        _, predicted = torch.max(outputs.data, 1) # torch.max는 각 예측에 대한 최대값과 그 위치(인덱스)를 반환. 위치만 필요하므로 _를 사용하여 최대값은 무시
        total += labels.size(0) # labels.size(0)는 현재 배치의 크기(샘플 수)
        correct += (predicted == labels).sum().item() # 일치하는 경우의 수를 텐서 형태로 반환하며, .item()으로 이를 파이썬의 스칼라 값으로 변환

print(f'10000개의 테스트 이미지에 대한 네트워크의 정확도: {100 * correct / total} %')

에폭 1, 훈련 손실: 0.6720112410709133, 검증 손실: 0.4400373150310673
검증 손실이 감소하였습니다. (inf 에서 0.440037로). 모델 저장 중...
에폭 2, 훈련 손실: 0.38876828034312133, 검증 손실: 0.3975578385499927
검증 손실이 감소하였습니다. (0.440037 에서 0.397558로). 모델 저장 중...
에폭 3, 훈련 손실: 0.3389457942579131, 검증 손실: 0.3321452489105577
검증 손실이 감소하였습니다. (0.397558 에서 0.332145로). 모델 저장 중...
에폭 4, 훈련 손실: 0.3108242120296552, 검증 손실: 0.32004278358136556
검증 손실이 감소하였습니다. (0.332145 에서 0.320043로). 모델 저장 중...
에폭 5, 훈련 손실: 0.2897479466550564, 검증 손실: 0.3214799569537887
에폭 6, 훈련 손실: 0.2747122911658438, 검증 손실: 0.3090512726183588
검증 손실이 감소하였습니다. (0.320043 에서 0.309051로). 모델 저장 중...
에폭 7, 훈련 손실: 0.2597498252504457, 검증 손실: 0.31585567985972135
에폭 8, 훈련 손실: 0.2506516980491284, 검증 손실: 0.3142875355743371
에폭 9, 훈련 손실: 0.23975225060625963, 검증 손실: 0.2828414982108867
검증 손실이 감소하였습니다. (0.309051 에서 0.282841로). 모델 저장 중...
에폭 10, 훈련 손실: 0.2282156703758278, 검증 손실: 0.30278751200618936
에폭 11, 훈련 손실: 0.22274764801408498, 검증 손실: 0.29372347010137606
에폭 12, 훈련 손실: 0.21306074037580758, 

<ipython-input-10-a542ed8fdbcc>:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('/content/drive/MyDrive/kdt_240424/m6_딥러닝/dataset/mode

10000개의 테스트 이미지에 대한 네트워크의 정확도: 89.44 %
